In [2]:
import csv
import pandas as pd
import json

def preprocess(ids, texts, indexes):
    reasult = {'id': [], 'text': [], 'class': []}
    if indexes != 'n/a':
        is_hate = json.loads(indexes)
    else:
        is_hate = []
    post_ids = [post_id_line[2:].strip() for post_id_line in ids.strip().split('\n')]
    posts = [text[2:].strip() for text in texts.strip().split('\n')]
    for index, row in enumerate(zip(post_ids, posts)):
        if len(row[1]) > 0:
            if row[0][0] == '.':
                new_post_id = row[0][1:].strip()
            else:
                new_post_id = row[0].strip()
            reasult['id'].append(new_post_id)
            if row[1][0] == '.':
                stripped_text = row[1][1:].strip()
            else:
                stripped_text = row[1].strip()
            reasult['text'].append(stripped_text)
            if index in is_hate:
                reasult['class'].append(1)
            else:
                reasult['class'].append(0)
    return pd.DataFrame.from_dict(reasult)


with open('hsd/Reddit/gab.csv') as csv_file:
    posts_df = pd.DataFrame(columns = ['id', 'text', 'class']) 
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)
    for row in csv_reader:
        posts_df = posts_df.append(preprocess(row[0], row[1], row[2]), ignore_index=True)
    posts_df.to_csv('hsd/Reddit/clean_gab.csv', index=False)
